In [1]:
import pandas as pd
import string
import re
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
lagu = pd.read_csv("song_df_indo.csv",encoding="latin")

Using TensorFlow backend.


In [2]:
def clean_doc(doc):
    doc = doc.replace("–"," ")
    doc = doc.lower()
    doc = re.sub('[^a-z ]+', '', doc)
    doc = re.sub('back to reff','', doc)
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word.lower() for word in tokens]
    return tokens

In [3]:
lagu.head()

,Unnamed: 0,id,judul,penyanyi,link,lirik,lang
0,8,9,A Lotta Love,3 Diva,https://lirik.kapanlagi.com/artis/3-diva/a-lot...,"(*) IÃ¢ÂÂ¦ I, I, I, IÃ¢ÂÂ¦ Need A Lotta Love...",id
1,10,11,A Minor,Kaneshiba,https://lirik.kapanlagi.com/artis/kaneshiba/a-...,Dari A minor Pindahnya ke D minor Masuk kunci ...,id
2,11,12,A N G,Naff,https://lirik.kapanlagi.com/artis/naff/a-n-g/,NAFF - A.N.G Seluruh hati tlah kudatangi Hanya...,id
3,17,18,A S A,Arwana,https://lirik.kapanlagi.com/artis/arwana/a-s-a/,Dalam malam kucoba mengerti Tentang luka yang ...,id
4,20,21,A Tribute To Maia,Tha Law,https://lirik.kapanlagi.com/artis/tha-law/a-tr...,Now throw ur hand in the air buat duo Maia Jar...,id


In [4]:
lagu["tokens"] = lagu["lirik"].apply(clean_doc)

In [5]:
lagu.head()

,Unnamed: 0,id,judul,penyanyi,link,lirik,lang,tokens
0,8,9,A Lotta Love,3 Diva,https://lirik.kapanlagi.com/artis/3-diva/a-lot...,"(*) IÃ¢ÂÂ¦ I, I, I, IÃ¢ÂÂ¦ Need A Lotta Love...",id,"[i, i, i, i, i, need, a, lotta, love, you, you..."
1,10,11,A Minor,Kaneshiba,https://lirik.kapanlagi.com/artis/kaneshiba/a-...,Dari A minor Pindahnya ke D minor Masuk kunci ...,id,"[dari, a, minor, pindahnya, ke, d, minor, masu..."
2,11,12,A N G,Naff,https://lirik.kapanlagi.com/artis/naff/a-n-g/,NAFF - A.N.G Seluruh hati tlah kudatangi Hanya...,id,"[naff, ang, seluruh, hati, tlah, kudatangi, ha..."
3,17,18,A S A,Arwana,https://lirik.kapanlagi.com/artis/arwana/a-s-a/,Dalam malam kucoba mengerti Tentang luka yang ...,id,"[dalam, malam, kucoba, mengerti, tentang, luka..."
4,20,21,A Tribute To Maia,Tha Law,https://lirik.kapanlagi.com/artis/tha-law/a-tr...,Now throw ur hand in the air buat duo Maia Jar...,id,"[now, throw, ur, hand, in, the, air, buat, duo..."


In [6]:
# organize into sequences of tokens
def create_seq(tokens):
    length = 20 + 1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    return sequences

In [7]:
lagu["sequences"] = lagu["tokens"].apply(create_seq)

In [8]:
lagu.head()

,Unnamed: 0,id,judul,penyanyi,link,lirik,lang,tokens,sequences
0,8,9,A Lotta Love,3 Diva,https://lirik.kapanlagi.com/artis/3-diva/a-lot...,"(*) IÃ¢ÂÂ¦ I, I, I, IÃ¢ÂÂ¦ Need A Lotta Love...",id,"[i, i, i, i, i, need, a, lotta, love, you, you...",[i i i i i need a lotta love you you you you y...
1,10,11,A Minor,Kaneshiba,https://lirik.kapanlagi.com/artis/kaneshiba/a-...,Dari A minor Pindahnya ke D minor Masuk kunci ...,id,"[dari, a, minor, pindahnya, ke, d, minor, masu...",[dari a minor pindahnya ke d minor masuk kunci...
2,11,12,A N G,Naff,https://lirik.kapanlagi.com/artis/naff/a-n-g/,NAFF - A.N.G Seluruh hati tlah kudatangi Hanya...,id,"[naff, ang, seluruh, hati, tlah, kudatangi, ha...",[naff ang seluruh hati tlah kudatangi hanya ha...
3,17,18,A S A,Arwana,https://lirik.kapanlagi.com/artis/arwana/a-s-a/,Dalam malam kucoba mengerti Tentang luka yang ...,id,"[dalam, malam, kucoba, mengerti, tentang, luka...",[dalam malam kucoba mengerti tentang luka yang...
4,20,21,A Tribute To Maia,Tha Law,https://lirik.kapanlagi.com/artis/tha-law/a-tr...,Now throw ur hand in the air buat duo Maia Jar...,id,"[now, throw, ur, hand, in, the, air, buat, duo...",[now throw ur hand in the air buat duo maia ja...


In [47]:
all_sequences = []
for seq in lagu["sequences"]:
    for i in seq:
        all_sequences.append(i)

In [48]:
lines = all_sequences

In [49]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [53]:
vocab_size = len(tokenizer.word_index) + 1

In [54]:
vocab_size

47808

In [56]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [57]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 20)            956160    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           48400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 47808)             4828608   
Total params: 5,923,668
Trainable params: 5,923,668
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
# compile moder
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=1024, epochs=5)

Epoch 1/5
  84992/1144134 [=>............................] - ETA: 1:04:24 - loss: 7.7428 - accuracy: 0.0244

KeyboardInterrupt: 